In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [4]:
df_train = pd.read_csv('../nss_capstone/df_train_07_20.csv').drop(columns='Unnamed: 0').dropna(subset=['ML'])

variables = ['Implied','Open Spread', 'Close Spread']  # Adjust the variables as per your dataset
target = 'W/L_bool'

X_train = df_train[variables]
y_train = df_train[target]

gbr = GradientBoostingClassifier(n_estimators=1000).fit(X_train, y_train)

df_val = pd.read_csv('../nss_capstone/df_val_21_22.csv').drop(columns='Unnamed: 0').dropna(subset=['ML'])  # Adjust the path and preprocessing as per your dataset

X_val = df_val[variables]
y_val = df_val[target]

predicted_probabilities = gbr.predict_proba(X_val)[:, 1]
closing_moneyline_odds = df_val.loc[X_val.index, 'Implied'].reset_index(drop=True)

percentage_differences = predicted_probabilities - closing_moneyline_odds    


payouts = df_val.loc[X_val.index, 'Payout'].reset_index(drop=True)
results = y_val.reset_index(drop = True)

threshold = 0.021


selected_bets = percentage_differences >= threshold

wager = 100  # Set the wager amount to $100

bets_won = selected_bets * results * payouts
bets_lost = selected_bets * (1-results) * -wager
profit = bets_won.sum() + bets_lost.sum()